# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [4]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [5]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [110]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
sfo_coordinates = pd.read_csv(file_path)
sfo_coordinates.rename(columns={'Neighborhood':'neighborhood'},inplace=True)

mean_neighbor_year=sfo_data.groupby(['year','neighborhood']).mean().reset_index()
mean_values_per_neighborhood=mean_neighbor_year[['neighborhood','sale_price_sqr_foot','housing_units','gross_rent']].groupby('neighborhood').mean().reset_index()
neighborhoods_with_loc=mean_values_per_neighborhood.merge(sfo_coordinates,how='inner',on='neighborhood')



In [111]:

def get_average(data,filter_col_list,filter_groupby_list):
    """Function to get averages from dataframemes """
    
    return data[filter_col_list].groupby(filter_groupby_list).mean().squeeze()
    
def plot_line_chart(data,x_label,y_label,title,color):
    """ function from real estate exercise to plot line charts"""
    empty_canvas=plt.figure()
    data_plot =data.plot.line(color=color)
    data_plot.set_xlabel(x_label)
    data_plot.set_ylabel(y_label)
    data_plot.set_title(title, fontweight="bold")

    return plt.close(empty_canvas)

def plot_bar(data,x_label,y_label,title_var):
    """ function from real estate exercise to plot line charts"""
    empty_canvas = plt.figure()
    bar_hist = data.plot.bar()
    ymin=data.min()-data.std()
    ymax=data.max()+data.std()

    bar_hist.set_ylim(ymin=ymin,ymax=ymax)
    bar_hist.set_xlabel(x_label)
    bar_hist.set_ylabel(y_label)
    bar_hist.set_title(title_var, fontweight="bold")

    return plt.close(empty_canvas)

def plot_hvplot_line(df,x_var,y_var,x_label,y_label,title,groupby):
   return df.hvplot.line(x=x_var,y=y_var,xlabel =x_label,ylabel =y_label,title=title,groupby=groupby)

def plot_hvplot_bar(df,x_var,y_var,x_label,y_label,title,rot=90,height=500,groupby=None):
      if groupby==None:
         empty_canvas =df.hvplot.bar(x=x_var,y=y_var,xlabel =x_label,ylabel =y_label,title=title,rot=rot,height=height)
      else:
         empty_canvas=df.hvplot.bar(x=x_var,y=y_var,xlabel =x_label,ylabel =y_label,title=title,rot=rot,height=height,groupby=groupby)
      return empty_canvas

def plot_mapbox_scatter(df,latitude,longitude,size_var,color_var,color_continuous_scale_var,   hover_name_VAR,
    title,   size_max_var=15,   zoom_var=11):

   canvas= px.scatter_mapbox(
      df,lat=latitude,lon=longitude,size=size_var,color=color_var,color_continuous_scale=color_continuous_scale_var,
      size_max=size_max_var,zoom=zoom_var,hover_name=hover_name_VAR,title=title
   )
   canvas.update_layout(mapbox_style="carto-positron")
   return canvas.show()




- - -

In [114]:
most_expensive=get_average(sfo_data,list(sfo_data.columns),['neighborhood']).reset_index().sort_values('sale_price_sqr_foot',ascending=False).head(10).reset_index(drop=True)

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [112]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    empty_canvas = plot_bar(get_average(sfo_data,['housing_units'],['year']),x_label='Year',y_label='Housing Units',title_var="Housing Units in San Francisco from 2010 to 2016")
      
    # YOUR CODE HERE!
    return pn.pane.Matplotlib(empty_canvas,tight=True)



def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    empty_canvas=plot_line_chart( get_average(sfo_data,['sale_price_sqr_foot','gross_rent'],['year'])['gross_rent']
            ,'Year','Average Gross rent',"Average Gross rent per Year",'red')
    
    # YOUR CODE HERE!
    return pn.pane.Matplotlib(empty_canvas,tight=True)



def average_sales_price():
    """Average Sales Price Per Year."""
    empty_canvas=plot_line_chart( get_average(sfo_data,['sale_price_sqr_foot','gross_rent'],['year'])['sale_price_sqr_foot'],
    'Year','Average Sqft Sale Price',"Average Price per Square footage per Year",'purple')
    
    # YOUR CODE HERE!
    return pn.pane.Matplotlib(empty_canvas,tight=True)


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    empty_canvas=    plot_hvplot_line(get_average(sfo_data,list(sfo_data.columns),['year', "neighborhood"]),
    x_var='year',y_var='sale_price_sqr_foot',x_label='Year',y_label='"Avg. Sale Price per Square Foot"',title='Average Prices by Neighborhood',groupby='neighborhood')
    # YOUR CODE HERE!
    return empty_canvas



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    most_expensive=get_average(sfo_data,list(sfo_data.columns),['neighborhood']).reset_index().sort_values('sale_price_sqr_foot',ascending=False).head(10).reset_index(drop=True)

    empty_canvas=plot_hvplot_bar(df=most_expensive,x_var='neighborhood',
    y_var='sale_price_sqr_foot',x_label='Neighborhood',y_label='Avg. Sale Price per SQFT',title='TOP 10 expensive neighborhoods')
    # YOUR CODE HERE!
    return empty_canvas


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""  
    mean_sales=get_average(sfo_data,list(sfo_data.columns),['year','neighborhood']).reset_index() 
    empty_canvas=plot_hvplot_bar(mean_sales,x_var='year', 
            y_var=['gross_rent','sale_price_sqr_foot'],
            x_label='Neighborhood',
            y_label='Num housing units',
            title='Top 10 Expensive Neighborhoods in SFO',
            groupby='neighborhood'
)
    # YOUR CODE HERE!
    return empty_canvas

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    most_expensive=get_average(sfo_data,list(sfo_data.columns),['neighborhood']).reset_index().sort_values('sale_price_sqr_foot',ascending=False).head(10).reset_index(drop=True)
    df_expensive_neighborhoods_per_year = mean_neighbor_year[mean_neighbor_year["neighborhood"].isin(most_expensive["neighborhood"])]

    empty_canvas=px.parallel_coordinates(most_expensive,color='sale_price_sqr_foot')

    # YOUR CODE HERE!
    return empty_canvas



def parallel_categories():
    """Parallel Categories Plot."""
    most_expensive=get_average(sfo_data,list(sfo_data.columns),['neighborhood']).reset_index().sort_values('sale_price_sqr_foot',ascending=False).head(10).reset_index(drop=True)
    df_expensive_neighborhoods_per_year = mean_neighbor_year[mean_neighbor_year["neighborhood"].isin(most_expensive["neighborhood"])]

    empty_canvas=px.parallel_categories(most_expensive,color='sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno)
    # YOUR CODE HERE!
    return empty_canvas



def neighborhood_map():
    """Neighborhood Map."""
    empty_canvas=plot_mapbox_scatter(df=neighborhoods_with_loc,latitude='Lat',longitude='Lon',size_var='sale_price_sqr_foot',color_var='gross_rent',
            color_continuous_scale_var=px.colors.cyclical.IceFire,hover_name_VAR='neighborhood',
            title='Average Sale Price Per SQFT and gross rent in San francisco'
        )
    # YOUR CODE HERE!
    return empty_canvas


def sunburst():
    """Sunburst Plot."""
    df_expensive_neighborhoods_per_year = mean_neighbor_year[mean_neighbor_year["neighborhood"].isin(most_expensive["neighborhood"])]
    empty_canvas=fig = px.sunburst(    df_expensive_neighborhoods_per_year,
    path=["year", "neighborhood"],
    values='sale_price_sqr_foot',
    color='gross_rent', 
    color_continuous_scale='Blues',
    title="Costs Analysis of Most Expensive neighborhoods in San Francisco per Year",
    height=700
    )
    # YOUR CODE HERE!
    return empty_canvas

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [115]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
title = pn.pane.Markdown(
    """
# Analysis of Real Estate  Market in San Francisco from 2010 to 2016
""",
    width=800,
)


# Create a tab layout for the dashboard
# YOUR CODE HERE!
welcome = pn.pane.Markdown(
    """
this dashboard provides visualizations that shows the evolution of the real estate market in the San Francisco Area
"""
)

# Create the dashboard
# YOUR CODE HERE!
tabs = pn.Tabs(
    ("Welcome", pn.Column(welcome, neighborhood_map())),
    ("Yearly Market Analysis", pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())),
    ("Neighborhood Analysis", pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())),
    ("Parallel Plots Analysis", pn.Column(parallel_coordinates(), parallel_categories(),width=960)),
    ("Sunburst Plot Analysis", pn.Column(sunburst())),
    )
dashboard = pn.Column(pn.Row(title), tabs, width=900)

C:\Users\ngond\anaconda3_1\envs\pyvizenv\lib\site-packages\matplotlib\axes\_base.py:3604: MatplotlibDeprecationWarning:


The `ymin` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `bottom` instead.

C:\Users\ngond\anaconda3_1\envs\pyvizenv\lib\site-packages\matplotlib\axes\_base.py:3610: MatplotlibDeprecationWarning:


The `ymax` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `top` instead.



## Serve the Panel Dashboard

In [116]:
# Serve the# dashboard
# YOUR CODE HERE!
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'31c5f987-8434-4a22-80e0-dbfdf14be355': {'defs': …

Column(width=900)
    [0] Row
        [0] Markdown(str, width=800)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Str(None)
        [1] Row
            [0] Matplotlib(None, tight=True)
            [1] Matplotlib(None, tight=True)
            [2] Matplotlib(None, tight=True)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
            [2] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
        [3] Column(width=960)
            [0] Plotly(Figure, visible=False)
            [1] Plotly(Figure, visible=False)
        [4] Column
            [0] Plotly(Figure, visible=False)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()